In [88]:
import os
import time
import pandas as pd
import simplekml
import shutil
import numpy as np
import socket
import datetime
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side
from openpyxl import load_workbook
from sys import platform
from pathlib import Path
import glob

# Vincenty Formula
from math import atan
from math import atan2
from math import cos
from math import radians
from math import sin
from math import sqrt
from math import tan

pd.options.mode.chained_assignment = None

# Measuring execution time
start_time = time.time()

# Specify Root Path

In [89]:
# Checks what operating system the user is using (Windows or OS)
if platform == 'win32':
    root_path = r'C:\Users\Stefanos\DeLasCasas CP, LLc\Ro De Las Casas - ' \
                r'DLC2P Server\AA Stefanos\Data Science\Python Projects\CIS ' \
                r'Data to Google Earth (Real Time)'
elif platform == 'darwin':
    root_path = r'/Users/Stefanos/Library/CloudStorage/' \
                r'OneDrive-SharedLibraries-DeLasCasasCP,LLc/Ro De Las Casas ' \
                r'- DLC2P Server/AA Stefanos/Data Science/Python Projects/' \
                r'CIS Data to Google Earth (Real Time)'

# Specify Variables

In [90]:
# SPECIFY ICONS
icon_on = 'https://img.icons8.com/material-sharp/24/1ff08a/give-way--v1.png'
icon_off = 'https://img.icons8.com/material-sharp/24/7950F2/give-way--v1.png'
icon_comments = 'https://img.icons8.com/ultraviolet/80/000000/comments.png'
icon_1200 = 'https://img.icons8.com/ios-filled/50/22C3E6/360-degrees.png'
icon_850 = 'https://img.icons8.com/ios-filled/50/F25081/360-degrees.png'
# SPECIFY SCALE FACTOR (Tallness of data points)
scale_factor = 175
# SPECIFY TRANSPARENCY (0 to 1)
transparency = 1
# SPECIFY ICON SCALE
icon_scale = 0.2
# SPECIFY DATA VISIBILITY WHEN OPENING KMZ FILE (True or False)
data_visibility = False
# SPECIFY COMPANY
company_name = 'Kinder Morgan'

# Vincenty Formula (0.5 mm Accuracy)

In [91]:
class VincentyInverse:
    def __init__(self, coord1, coord2, maxiter=200, tol=10**-12):

        #--- CONSTANTS ------------------------------------+

        a = 6378137.0                    # radius at equator in meters (WGS-84)
        f = 1/298.257223563              # flattening of the ellipsoid (WGS-84)
        b = (1-f)*a

        phi_1, lon_1, = coord1                      # (lat=L_?,lon=phi_?)
        phi_2, lon_2, = coord2

        u_1 = atan((1-f)*tan(radians(phi_1)))
        u_2 = atan((1-f)*tan(radians(phi_2)))

        L = radians(lon_2-lon_1)

        Lambda = L                           # set initial value of lambda to L

        sin_u1 = sin(u_1)
        cos_u1 = cos(u_1)
        sin_u2 = sin(u_2)
        cos_u2 = cos(u_2)

        #--- BEGIN ITERATIONS -----------------------------+
        self.iters = 0
        for i in range(0, maxiter):
            self.iters += 1

            cos_lambda = cos(Lambda)
            sin_lambda = sin(Lambda)
            sin_sigma = sqrt((cos_u2*sin(Lambda))**2+
                             (cos_u1*sin_u2-sin_u1*cos_u2*cos_lambda)**2)
            cos_sigma = sin_u1*sin_u2+cos_u1*cos_u2*cos_lambda
            sigma = atan2(sin_sigma,cos_sigma)
            sin_alpha = (cos_u1*cos_u2*sin_lambda)/sin_sigma
            cos_sq_alpha = 1-sin_alpha**2
            cos2_sigma_m = cos_sigma-((2*sin_u1*sin_u2)/cos_sq_alpha)
            C = (f/16)*cos_sq_alpha*(4+f*(4-3*cos_sq_alpha))
            Lambda_prev = Lambda
            Lambda = L+(1-C)*f*sin_alpha*(sigma+C*sin_sigma*
                                          (cos2_sigma_m+C*cos_sigma*
                                           (-1+2*cos2_sigma_m**2)))

            # Successful convergence
            diff = abs(Lambda_prev-Lambda)
            if diff <= tol:
                break

        u_sq = cos_sq_alpha*((a**2-b**2)/b**2)
        A = 1+(u_sq/16384)*(4096+u_sq*(-768+u_sq*(320-175*u_sq)))
        B = (u_sq/1024)*(256+u_sq*(-128+u_sq*(74-47*u_sq)))
        delta_sig = B*sin_sigma*(cos2_sigma_m+0.25*B*
                                 (cos_sigma*(-1+2*cos2_sigma_m**2)-
                                  (1/6)*B*cos2_sigma_m*(-3+4*sin_sigma**2)*
                                  (-3+4*cos2_sigma_m**2)))

        self.m = b*A*(sigma-delta_sig)         # output distance in meters
        #self.km = self.m/1000                 # output distance in kilometers
        #self.mm = self.m*1000                 # output distance in millimeters
        self.miles = self.m*0.000621371        # output distance in miles
        self.ft = self.miles*5280              # output distance in feet
        #self.inches = self.ft*12              # output distance in inches
        #self.yards = self.ft/3                # output distance in yards

# if __name__ == "__VincentyInverse__":
#     main()

# Data Cleaning

In [92]:
# List of files contained in the original data folder
if platform == 'win32':
    raw_data_list = os.listdir(r'/Users/Stefanos/'
                               r'DeLasCasas CP, LLc/'
                               r'Ro De Las Casas - DLC2P Server/AA Stefanos/'
                               r'Data Science/Python Projects/'
                               r'CIS Data to Google Earth (Real Time)/'
                               r'Original Data')
elif platform == 'darwin':
    raw_data_list = os.listdir(r'/Users/Stefanos/Library/CloudStorage/'
                               r'OneDrive-SharedLibraries-DeLasCasasCP,LLc/'
                               r'Ro De Las Casas - DLC2P Server/AA Stefanos/'
                               r'Data Science/Python Projects/'
                               r'CIS Data to Google Earth (Real Time)/'
                               r'Original Data')

In [93]:
# Replaces data folder with an empty one
if os.path.exists(root_path + r'/Data/'):
    shutil.rmtree(root_path + r'/Data/')
    os.makedirs(root_path + r'/Data/')
if not os.path.exists(root_path + r'/Data/'):
    os.makedirs(root_path + r'/Data/')

In [94]:
# Remove log file if it exists
if os.path.exists(root_path + str(Path(r'/Log.txt'))):
    os.remove(root_path + str(Path(r'/Log.txt')))
# Creates a log file to track information of interest
with open(root_path + str(Path(r'/Log.txt')), 'a') as f:
    f.write('Rows dropped with missing GPS points:')
    f.write('\n')

In [95]:
# List to exclude certain files
filtered_data_list = []
i = 0       # List counter for files

# Filters the folder and excludes files that start with '.'
while i < len(raw_data_list):
    if not raw_data_list[i][0] == '.':
        filtered_data_list.append(raw_data_list[i])
    i += 1

In [96]:
j = 0       # List counter for '.csv' files

# Removes '.DAT' string from any files that have it
while j < len(filtered_data_list):
    original_name = root_path + str(Path(r'/Original Data/' +
                                         filtered_data_list[j]))
    filtered_data_list[j] = filtered_data_list[j].replace('.DAT', '')
    new_name = root_path + str(Path(r'/Original Data/' + filtered_data_list[j]))
    os.rename(original_name, new_name)
    j += 1

In [97]:
# List for total miles per '.csv' file
total_miles_list = []

total_rows_dropped = []
i = 0       # Loop counter for calculating distance (ft)
j = 0       # List counter for csv files

# Goes through all the files in the list
while j < len(filtered_data_list):
    # Create dataframe from a csv file
    df_cis = pd.read_csv(root_path +
                         str(Path(r'/Original Data/' +
                                  filtered_data_list[j])))
    # Drops elevation and PDOP columns
    df_cis = df_cis.iloc[:, :6]
    # Delete rows that CIS was skipped
    df_cis = df_cis.loc[df_cis['On Potential'] != 'SKIP'].reset_index(drop=True)
    # Extract starting stationing number (ft) from '.csv' title
    starting_stationing = float(filtered_data_list[j].split()[1].split('SN')[1]\
        .replace('+',''))
    # Create new column
    df_cis['Stationing (ft)'] = 0
    # Convert object columns to numbers
    df_cis[['On Potential', 'Off Potential']] = \
                            df_cis[['On Potential', 'Off Potential']]\
                            .apply(pd.to_numeric, errors='coerce', axis=1)
    df_cis[['On Potential', 'Off Potential']] /= 1000
    # Trim white space from comments
    df_cis['Comments'] = df_cis['Comments'].str.strip()
    # Create new column
    df_cis['Distance (ft)'] = 0
    last_index2 = df_cis.last_valid_index()
    # Drop rows that don't have GPS coordinates
    df_cis['Latitude'].replace('', np.nan, inplace=True)
    df_cis.dropna(subset=['Latitude'], inplace=True)
    df_cis.reset_index(drop=True, inplace=True)
    last_index = df_cis.last_valid_index()
    total_rows_dropped.append(last_index2-last_index)
    # Record number of rows dropped for each file
    with open(root_path + str(Path(r'/Log.txt')), 'a') as f:
        f.write('- {} ({})'.format(filtered_data_list[j],
                                  total_rows_dropped[j]))
        f.write('\n')

    while i < last_index:
        # Infers actual stationing number (xxxx+xx)
        df_cis['Stationing (ft)'][i] = starting_stationing+\
                                       df_cis['Station'][i]
        df_cis['Stationing (ft)'][i] = str(df_cis['Stationing (ft)'][i])\
            .split('.')[0]
        string_length = len(df_cis['Stationing (ft)'][i])
        df_cis['Stationing (ft)'][i] = df_cis['Stationing (ft)'][i]\
                                             [:string_length-2] + '+' + \
                                             df_cis['Stationing (ft)'][i]\
                                             [string_length-2:]
        # Calculates distance between GPS coordinates and stationing (ft)
        try:
            df_cis['Distance (ft)'][i+1] = \
            VincentyInverse([df_cis['Latitude'][i],
                             df_cis['Longitude'][i]],
                            [df_cis['Latitude'][i+1],
                             df_cis['Longitude'][i+1]]).ft
        except ZeroDivisionError:
            df_cis['Distance (ft)'][i+1] = 0
        i += 1
    # Calculate stationing (ft) value of last row
    df_cis['Stationing (ft)'][i] = starting_stationing+\
                                       df_cis['Station'][i]
    df_cis['Stationing (ft)'][i] = str(df_cis['Stationing (ft)'][i])\
        .split('.')[0]
    string_length = len(df_cis['Stationing (ft)'][i])
    df_cis['Stationing (ft)'][i] = df_cis['Stationing (ft)'][i]\
                                         [:string_length-2] + '+' + \
                                         df_cis['Stationing (ft)'][i]\
                                         [string_length-2:]
    # Records total miles in a list
    total_miles_list.append(max(df_cis['Station'])/5280)
    df_cis.to_csv(root_path +
                  str(Path(r'/Data/' +
                           filtered_data_list[j])), index=False)
    i = 0
    j += 1

# Create Google Earth Folders

In [98]:
# Replaces output folder with an empty one
if os.path.exists(root_path + r'/Output/'):
    shutil.rmtree(root_path + r'/Output/')
    os.makedirs(root_path + r'/Output/')
if not os.path.exists(root_path + r'/Output/'):
    os.makedirs(root_path + r'/Output/')

In [99]:
cis_kml =[]
type_folders = []
current_mile = 1
k = 0       # List counter for type folders
j = 0       # List counter for '.csv' files
i = 0       # List counter for '.kml' files based on miles

In [100]:
# Create CIS survey '.kml' files per mile #
while j < len(total_miles_list):
    miles_remaining = round(total_miles_list[j], 2)
    # Create type folders
    while miles_remaining > 0:
        cis_kml.append(filtered_data_list[j].split('.csv', 1)[0] +
                       ' (Mile ' + current_mile.__str__() + ')')
        cis_kml[i] = simplekml.Kml()
        type_folders.append(filtered_data_list[j].split('.CSV', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(On)')
        type_folders.append(filtered_data_list[j].split('.CSV', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(Off)')
        type_folders.append(filtered_data_list[j].split('.CSV', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(Comments)')
        type_folders.append(filtered_data_list[j].split('.CSV', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(-1.2 V)')
        type_folders.append(filtered_data_list[j].split('.CSV', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(-0.85 V)')
        type_folders[k] = cis_kml[i].newfolder(name='On')
        type_folders[k+1] = cis_kml[i].newfolder(name='Off')
        type_folders[k+2] = cis_kml[i].newfolder(name='Comments')
        type_folders[k+3] = cis_kml[i].newfolder(name='-1.2 V')
        type_folders[k+4] = cis_kml[i].newfolder(name='-0.85 V')
        cis_kml[i].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.CSV', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)
        i += 1
        k += 5
    j += 1
    current_mile = 1

In [101]:
# Duplicated GPS coordinates
#df = df_cis_on[df_cis_on.duplicated(subset=['Longitude', 'Latitude'],
                                    #keep=False)]

# CIS Data

## On

In [102]:
feet_counter = 5280
style = simplekml.Style()
i = 0       # Loop counter for rows
j = 0       # Loop counter for '.CSV' files
k = 0       # Loop counter for type folders (0 = 'On')
kml_file = 0        #

# Goes through all '.CSV' files
while j < len(filtered_data_list):
    df_cis_on = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_on = df_cis_on[df_cis_on['On Potential'] != 0]
    df_cis_on = df_cis_on[['Station', 'Stationing (ft)', 'Longitude',
                           'Latitude', 'On Potential']].reset_index(drop=True)
    df_cis_on['On Potential'] = df_cis_on['On Potential']*(-1)
    last_index = df_cis_on.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_on.loc[i, 'Station'] < feet_counter:
            pnt = type_folders[k]\
                .newpoint(name=df_cis_on.loc[i, 'On Potential']*(-1),
                          visibility=data_visibility)
            pnt.style.balloonstyle.text = \
                'Potential: -{} V\nLongitude: {}\nLatitude: {}\n'\
                'Station (ft): {}'\
                    .format(df_cis_on.loc[i, 'On Potential'],
                            df_cis_on.loc[i, 'Longitude'],
                            df_cis_on.loc[i, 'Latitude'],
                            df_cis_on.loc[i, 'Stationing (ft)'])
            pnt.coords = [(df_cis_on.loc[i, 'Longitude'],
                           df_cis_on.loc[i, 'Latitude'],
                           df_cis_on.loc[i, 'On Potential']*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_on
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.CSV', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

## Off

In [103]:
k = 1       # Loop counter for type folders (1 = 'Off')

# Goes through all '.CSV' files
while j < len(filtered_data_list):
    df_cis_off = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_off = df_cis_off[df_cis_off['Off Potential'] != 0]
    df_cis_off = df_cis_off[['Station', 'Stationing (ft)',
                             'Longitude', 'Latitude',
                             'Off Potential']].reset_index(drop=True)
    df_cis_off['Off Potential'] = df_cis_off['Off Potential']*(-1)
    last_index = df_cis_off.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_off.loc[i, 'Station'] < feet_counter:
            pnt = type_folders[k]\
                .newpoint(name=df_cis_off.loc[i, 'Off Potential']*(-1),
                          visibility=data_visibility)
            pnt.style.balloonstyle.text = \
                'Potential: -{} V\nLongitude: {}\nLatitude: {}\n'\
                'Station (ft): {}'\
                    .format(df_cis_off.loc[i, 'Off Potential'],
                            df_cis_off.loc[i, 'Longitude'],
                            df_cis_off.loc[i, 'Latitude'],
                            df_cis_off.loc[i, 'Stationing (ft)'])
            pnt.coords = [(df_cis_off.loc[i, 'Longitude'],
                           df_cis_off.loc[i, 'Latitude'],
                           df_cis_off.loc[i, 'Off Potential']*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_off
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.CSV', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

## Comments

In [104]:
k = 2       # Loop counter for type folders (2 = 'Comments')

# Goes through all '.CSV' files
while j < len(filtered_data_list):
    df_cis_comments = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_comments.dropna(inplace=True)
    df_cis_comments.reset_index(drop=True, inplace=True)
    last_index = df_cis_comments.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_comments.loc[i, 'Station'] < \
                feet_counter:
            pnt = type_folders[k]\
                .newpoint(name=df_cis_comments.loc[i,'Comments'],
                          visibility=data_visibility)
            if df_cis_comments.loc[i, 'On Potential'] == 0:
                potential = df_cis_comments.loc[i, 'Off Potential'].__str__()\
                            + ' V (Off)'
            else:
                potential = df_cis_comments.loc[i, 'On Potential'].__str__() \
                            + ' V (On)'
            pnt.style.balloonstyle.text = \
                'Potential: {}\nLongitude: {}\nLatitude: {}\n' \
                'Station (ft): {}\nComment: {}'\
                    .format(potential,
                            df_cis_comments.loc[i, 'Longitude'],
                            df_cis_comments.loc[i, 'Latitude'],
                            df_cis_comments.loc[i, 'Stationing (ft)'],
                            df_cis_comments.loc[i, 'Comments'])
            pnt.coords = [(df_cis_comments.loc[i, 'Longitude'],
                           df_cis_comments.loc[i, 'Latitude'],
                           0)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_comments
            pnt.style.iconstyle.scale = icon_scale*1.5
            pnt.style.labelstyle.scale = 0.5
            #pnt.extrude = 0
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.CSV', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

## -1.2 V

In [105]:
k = 3       # Loop counter for type folders (3 = '-1.2 V')

# Goes through all '.CSV' files
while j < len(filtered_data_list):
    df_cis_1200 = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    last_index = df_cis_1200.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_1200.loc[i, 'Station'] < \
                feet_counter:
            pnt = type_folders[k].newpoint(name='-1.200',
                                           visibility=data_visibility)
            pnt.style.balloonstyle.text = 'Potential: -1.2 V'
            pnt.coords = [(df_cis_1200.loc[i, 'Longitude'],
                           df_cis_1200.loc[i, 'Latitude'],
                           1.2*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_1200
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.CSV', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

## -0.85 V

In [106]:
k = 4       # Loop counter for type folders (4 = '-0.85 V')

# Goes through all '.CSV' files
while j < len(filtered_data_list):
    df_cis_850 = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    last_index = df_cis_850.last_valid_index()
    miles_remaining = round(total_miles_list[j], 2)
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_850.loc[i, 'Station'] < \
                feet_counter:
            pnt = type_folders[k].newpoint(name='-0.850',
                                           visibility=data_visibility)
            pnt.style.balloonstyle.text = 'Potential: -0.85 V'
            pnt.coords = [(df_cis_850.loc[i, 'Longitude'],
                           df_cis_850.loc[i, 'Latitude'],
                           0.85*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_850
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 1
            pnt.style.linestyle.width = 0.01
            pnt.style.linestyle.color = \
                simplekml.Color.rgb(255, 255, 255, round(255*0.15))
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.CSV', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = round(total_miles_list[j], 2)
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

# Areas of Interest

## Less Negative than -0.85V "On"

In [107]:
# Combines all '.CSV' files at specified folder
df_cis_filtered = pd.concat(map(pd.read_csv,
                                glob.glob(root_path + r'/Data/*.CSV')),
                            ignore_index = True)
df_cis_filtered = df_cis_filtered[df_cis_filtered['On Potential'] != 0]
df_cis_filtered = df_cis_filtered[['Station', 'Stationing (ft)', 'Longitude',
                                   'Latitude',
                                   'On Potential']].reset_index(drop=True)
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [108]:
i = 1       # Loop counter for rows

# Finds crossing points
while i < last_index:
    if ((df_cis_filtered.loc[i-1, 'On Potential'] > -0.85) and
        (df_cis_filtered.loc[i, 'On Potential'] < -0.85) and
        (df_cis_filtered.loc[i+1, 'On Potential'] > -0.85) and
        i >= 1):
        df_cis_filtered.loc[i, 'Crossing Point'] = 'XX'
    elif ((df_cis_filtered.loc[i+1, 'On Potential'] > -0.85) and
        (df_cis_filtered.loc[i, 'On Potential'] < -0.85)):
        df_cis_filtered.loc[i, 'Crossing Point'] = 'X'
    elif ((df_cis_filtered.loc[i+1, 'On Potential'] < -0.85) and
          (df_cis_filtered.loc[i, 'On Potential'] > -0.85)):
        df_cis_filtered.loc[i+1, 'Crossing Point'] = 'X'
    i += 1
df_cis_filtered = df_cis_filtered[df_cis_filtered['Crossing Point'] != '']\
    .reset_index(drop=True)

In [109]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
        .repeat(df_cis_filtered['Crossing Point']
                .isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index()+1)/2)
    # Create report dataframe
    df_cis_report = pd.DataFrame(index=np.arange(last_index),
                                 columns=['Station', 'Station Number',
                                          'Latitude', 'Longitude',
                                          'Station', 'Station Number',
                                          'Latitude', 'Longitude'])
    df_cis_report['Length (ft)'] = 0.0
    df_cis_report['Comments'] = ''

    i = 0       #
    j = 0       #

    # Structure data
    while j < int((df_cis_filtered.last_valid_index()+1)/2):
        # Start
        df_cis_report.iloc[j, 0] = df_cis_filtered.loc[i, 'Station']
        df_cis_report.iloc[j, 1] = df_cis_filtered.loc[i, 'Stationing (ft)']
        df_cis_report.iloc[j, 2] = df_cis_filtered.loc[i, 'Latitude']
        df_cis_report.iloc[j, 3] = df_cis_filtered.loc[i, 'Longitude']
        # End
        df_cis_report.iloc[j, 4] = df_cis_filtered.loc[i+1, 'Station']
        df_cis_report.iloc[j, 5] = df_cis_filtered.loc[i+1, 'Stationing (ft)']
        df_cis_report.iloc[j, 6] = df_cis_filtered.loc[i+1, 'Latitude']
        df_cis_report.iloc[j, 7] = df_cis_filtered.loc[i+1, 'Longitude']
        # Length
        df_cis_report.iloc[j, 8] = df_cis_report.iloc[j, 4] - \
                                   df_cis_report.iloc[j, 0]
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)
    # Saves to excel
    with pd.ExcelWriter(root_path + r'/Output/CIS Exception Report (' +
                        filtered_data_list[0].split()[0] + ').xlsx',
                        mode='w', engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4,
                               sheet_name='Less Negative than -0.85V (On)',
                               index=False)

In [110]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(root_path + r'/Output/CIS Exception Report (' +
                             filtered_data_list[0].split()[0] + ').xlsx')
    sheet = wb['Less Negative than -0.85V (On)']
    # Company name with pipeline ID
    sheet.cell(1, 1).value = company_name + ' (' + \
                             filtered_data_list[0].split()[0] + ')'
    # Station numbers surveyed
    sheet.cell(2, 1).value = filtered_data_list[0].split()[1] + ' to ' + \
                             filtered_data_list[0].split()[3].split('.CSV')[0]
    # Start
    sheet.cell(4, 1).value = 'Start'
    # End
    sheet.cell(4, 4).value = 'End'
    # Total pipeline distance with issues
    sheet.cell(last_index+7, 1).value = 'Total pipeline distance ' \
                                        'surveyed = ' + \
                                        str(sum(total_miles_list*5280)) + \
                                        ' feet or ' + \
                                        str(round(sum(total_miles_list), 2)) + \
                                        ' miles'
    sheet.cell(last_index+8, 1).value = 'Total pipeline distance less ' \
                                        'negative than -0.85V "On" = ' + \
                                        str(df_cis_report['Length (ft)']
                                            .sum()) + ' feet (' + \
                                        str(round(df_cis_report['Length (ft)']
                                                  .sum()/\
                                        sum(total_miles_list*5280), 2)) + ' %)'

In [111]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(last_index+7, 1).font = Font(bold=True)
    sheet.cell(last_index+8, 1).font = Font(bold=True)
    # Merging
    sheet.merge_cells('A1:H1')
    sheet.merge_cells('A2:H2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')
    # Alignment
    i = 0

    while i < (last_index+5):
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
        i += 1
    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:H' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)
            i += 1
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        i += 1
    # Column widths
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 15
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 50

In [112]:
if df_cis_filtered.shape[0] != 0:
    wb.save(root_path + r'/Output/CIS Exception Report (' +
            filtered_data_list[0].split()[0] + ').xlsx')

## Less Negative than -0.85V "Off"

In [113]:
# Combines all '.CSV' files at specified folder
df_cis_filtered = pd.concat(map(pd.read_csv,
                                glob.glob(root_path + r'/Data/*.CSV')),
                            ignore_index = True)
df_cis_filtered = df_cis_filtered[df_cis_filtered['Off Potential'] != 0]
df_cis_filtered = df_cis_filtered[['Station', 'Stationing (ft)', 'Longitude',
                                   'Latitude',
                                   'Off Potential']].reset_index(drop=True)
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [114]:
i = 1       # Loop counter for rows

# Finds crossing points
while i < last_index:
    if ((df_cis_filtered.loc[i-1, 'Off Potential'] > -0.85) and
        (df_cis_filtered.loc[i, 'Off Potential'] < -0.85) and
        (df_cis_filtered.loc[i+1, 'Off Potential'] > -0.85) and
        i >= 1):
        df_cis_filtered.loc[i, 'Crossing Point'] = 'XX'
    elif ((df_cis_filtered.loc[i+1, 'Off Potential'] > -0.85) and
        (df_cis_filtered.loc[i, 'Off Potential'] < -0.85)):
        df_cis_filtered.loc[i, 'Crossing Point'] = 'X'
    elif ((df_cis_filtered.loc[i+1, 'Off Potential'] < -0.85) and
          (df_cis_filtered.loc[i, 'Off Potential'] > -0.85)):
        df_cis_filtered.loc[i+1, 'Crossing Point'] = 'X'
    i += 1
df_cis_filtered = df_cis_filtered[df_cis_filtered['Crossing Point'] != '']\
    .reset_index(drop=True)

In [115]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
        .repeat(df_cis_filtered['Crossing Point']
                .isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index()+1)/2)
    # Create report dataframe
    df_cis_report = pd.DataFrame(index=np.arange(last_index),
                                 columns=['Station', 'Station Number',
                                          'Latitude', 'Longitude',
                                          'Station', 'Station Number',
                                          'Latitude', 'Longitude'])
    df_cis_report['Length (ft)'] = 0.0
    df_cis_report['Comments'] = ''

    i = 0       #
    j = 0       #

    # Structure data
    while j < int((df_cis_filtered.last_valid_index()+1)/2):
        # Start
        df_cis_report.iloc[j, 0] = df_cis_filtered.loc[i, 'Station']
        df_cis_report.iloc[j, 1] = df_cis_filtered.loc[i, 'Stationing (ft)']
        df_cis_report.iloc[j, 2] = df_cis_filtered.loc[i, 'Latitude']
        df_cis_report.iloc[j, 3] = df_cis_filtered.loc[i, 'Longitude']
        # End
        df_cis_report.iloc[j, 4] = df_cis_filtered.loc[i+1, 'Station']
        df_cis_report.iloc[j, 5] = df_cis_filtered.loc[i+1, 'Stationing (ft)']
        df_cis_report.iloc[j, 6] = df_cis_filtered.loc[i+1, 'Latitude']
        df_cis_report.iloc[j, 7] = df_cis_filtered.loc[i+1, 'Longitude']
        # Length
        df_cis_report.iloc[j, 8] = df_cis_report.iloc[j, 4] - \
                                   df_cis_report.iloc[j, 0]
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)
    # Saves to excel
    writing_mode = 'w'
    if os.path.exists(root_path + r'/Output/CIS Exception Report (' +
                      filtered_data_list[0].split()[0] + ').xlsx'):
        writing_mode = 'a'
    with pd.ExcelWriter(root_path + r'/Output/CIS Exception Report (' +
                        filtered_data_list[0].split()[0] + ').xlsx',
                        mode=writing_mode, engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4,
                               sheet_name='Less Negative than -0.85V (Off)',
                               index=False)

In [116]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(root_path + r'/Output/CIS Exception Report (' +
                             filtered_data_list[0].split()[0] + ').xlsx')
    sheet = wb['Less Negative than -0.85V (Off)']
    # Company name with pipeline ID
    sheet.cell(1, 1).value = company_name + ' (' + \
                             filtered_data_list[0].split()[0] + ')'
    # Station numbers surveyed
    sheet.cell(2, 1).value = filtered_data_list[0].split()[1] + ' to ' + \
                             filtered_data_list[0].split()[3].split('.CSV')[0]
    # Start
    sheet.cell(4, 1).value = 'Start'
    # End
    sheet.cell(4, 4).value = 'End'
    # Total pipeline distance with issues
    sheet.cell(last_index+7, 1).value = 'Total pipeline distance ' \
                                        'surveyed = ' + \
                                        str(sum(total_miles_list*5280)) + \
                                        ' feet or ' + \
                                        str(round(sum(total_miles_list), 2)) + \
                                        ' miles'
    sheet.cell(last_index+8, 1).value = 'Total pipeline distance less ' \
                                        'negative than -0.85V "Off" = ' + \
                                        str(df_cis_report['Length (ft)']
                                            .sum()) + ' feet (' + \
                                        str(round(df_cis_report['Length (ft)']
                                                  .sum()/\
                                        sum(total_miles_list*5280), 2)) + ' %)'

In [117]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(last_index+7, 1).font = Font(bold=True)
    sheet.cell(last_index+8, 1).font = Font(bold=True)
    # Merging
    sheet.merge_cells('A1:H1')
    sheet.merge_cells('A2:H2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')
    # Alignment
    i = 0

    while i < (last_index+5):
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
        i += 1
    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:H' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)
            i += 1
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        i += 1
    # Column widths
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 15
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 50

In [118]:
if df_cis_filtered.shape[0] != 0:
    wb.save(root_path + r'/Output/CIS Exception Report (' +
            filtered_data_list[0].split()[0] + ').xlsx')

## More Negative than -1.2V "Off"

In [119]:
# Combines all '.CSV' files at specified folder
df_cis_filtered = pd.concat(map(pd.read_csv,
                                glob.glob(root_path + r'/Data/*.CSV')),
                            ignore_index = True)
df_cis_filtered = df_cis_filtered[df_cis_filtered['Off Potential'] != 0]
df_cis_filtered = df_cis_filtered[['Station', 'Stationing (ft)', 'Longitude',
                                   'Latitude',
                                   'Off Potential']].reset_index(drop=True)
df_cis_filtered['Crossing Point'] = ''
last_index = df_cis_filtered.last_valid_index()

In [120]:
i = 1       # Loop counter for rows

# Finds crossing points
while i < last_index:
    if ((df_cis_filtered.loc[i-1, 'Off Potential'] < -1.2) and
        (df_cis_filtered.loc[i, 'Off Potential'] > -1.2) and
        (df_cis_filtered.loc[i+1, 'Off Potential'] < -1.2) and
        i >= 1):
        df_cis_filtered.loc[i, 'Crossing Point'] = 'XX'
    elif ((df_cis_filtered.loc[i+1, 'Off Potential'] < -1.2) and
        (df_cis_filtered.loc[i, 'Off Potential'] > -1.2)):
        df_cis_filtered.loc[i, 'Crossing Point'] = 'X'
    elif ((df_cis_filtered.loc[i+1, 'Off Potential'] > -1.2) and
          (df_cis_filtered.loc[i, 'Off Potential'] < -1.2)):
        df_cis_filtered.loc[i+1, 'Crossing Point'] = 'X'
    i += 1
df_cis_filtered = df_cis_filtered[df_cis_filtered['Crossing Point'] != '']\
    .reset_index(drop=True)

In [121]:
if df_cis_filtered.shape[0] != 0:
    # Replicates rows that have 'XX'
    df_cis_filtered = df_cis_filtered.loc[df_cis_filtered.index
        .repeat(df_cis_filtered['Crossing Point']
                .isin(['XX']).add(1))].reset_index(drop=True)
    last_index = int((df_cis_filtered.last_valid_index()+1)/2)
    # Create report dataframe
    df_cis_report = pd.DataFrame(index=np.arange(last_index),
                                 columns=['Station', 'Station Number',
                                          'Latitude', 'Longitude',
                                          'Station', 'Station Number',
                                          'Latitude', 'Longitude'])
    df_cis_report['Length (ft)'] = 0.0
    df_cis_report['Comments'] = ''

    i = 0       #
    j = 0       #

    # Structure data
    while j < int((df_cis_filtered.last_valid_index()+1)/2):
        # Start
        df_cis_report.iloc[j, 0] = df_cis_filtered.loc[i, 'Station']
        df_cis_report.iloc[j, 1] = df_cis_filtered.loc[i, 'Stationing (ft)']
        df_cis_report.iloc[j, 2] = df_cis_filtered.loc[i, 'Latitude']
        df_cis_report.iloc[j, 3] = df_cis_filtered.loc[i, 'Longitude']
        # End
        df_cis_report.iloc[j, 4] = df_cis_filtered.loc[i+1, 'Station']
        df_cis_report.iloc[j, 5] = df_cis_filtered.loc[i+1, 'Stationing (ft)']
        df_cis_report.iloc[j, 6] = df_cis_filtered.loc[i+1, 'Latitude']
        df_cis_report.iloc[j, 7] = df_cis_filtered.loc[i+1, 'Longitude']
        # Length
        df_cis_report.iloc[j, 8] = df_cis_report.iloc[j, 4] - \
                                   df_cis_report.iloc[j, 0]
        j += 1
        i += 2

    # Deletes station columns
    df_cis_report = df_cis_report.drop(df_cis_report.iloc[:, [0, 4]], axis=1)
    # Saves to excel
    writing_mode = 'w'
    if os.path.exists(root_path + r'/Output/CIS Exception Report (' +
                      filtered_data_list[0].split()[0] + ').xlsx'):
        writing_mode = 'a'
    with pd.ExcelWriter(root_path + r'/Output/CIS Exception Report (' +
                        filtered_data_list[0].split()[0] + ').xlsx',
                        mode=writing_mode, engine='openpyxl') as writer:
        df_cis_report.to_excel(writer, startrow=4,
                               sheet_name='More Negative than -1.2V (Off)',
                               index=False)

In [122]:
if df_cis_filtered.shape[0] != 0:
    wb = load_workbook(root_path + r'/Output/CIS Exception Report (' +
                             filtered_data_list[0].split()[0] + ').xlsx')
    sheet = wb['More Negative than -1.2V (Off)']
    # Company name with pipeline ID
    sheet.cell(1, 1).value = company_name + ' (' + \
                             filtered_data_list[0].split()[0] + ')'
    # Station numbers surveyed
    sheet.cell(2, 1).value = filtered_data_list[0].split()[1] + ' to ' + \
                             filtered_data_list[0].split()[3].split('.CSV')[0]
    # Start
    sheet.cell(4, 1).value = 'Start'
    # End
    sheet.cell(4, 4).value = 'End'
    # Total pipeline distance with issues
    sheet.cell(last_index+7, 1).value = 'Total pipeline distance ' \
                                        'surveyed = ' + \
                                        str(sum(total_miles_list*5280)) + \
                                        ' feet or ' + \
                                        str(round(sum(total_miles_list), 2)) + \
                                        ' miles'
    sheet.cell(last_index+8, 1).value = 'Total pipeline distance more ' \
                                        'negative than -1.2V "Off" = ' + \
                                        str(df_cis_report['Length (ft)']
                                            .sum()) + ' feet (' + \
                                        str(round(df_cis_report['Length (ft)']
                                                  .sum()/\
                                        sum(total_miles_list*5280), 2)) + ' %)'

In [123]:
if df_cis_filtered.shape[0] != 0:
    # Font
    sheet.cell(1, 1).font = Font(size=14, bold=True)
    sheet.cell(2, 1).font = Font(italic=True)
    sheet.cell(4, 1).font = Font(bold=True)
    sheet.cell(4, 4).font = Font(bold=True)
    sheet.cell(4, 7).font = Font(bold=True)
    sheet.cell(4, 8).font = Font(bold=True)
    sheet.cell(last_index+7, 1).font = Font(bold=True)
    sheet.cell(last_index+8, 1).font = Font(bold=True)
    # Merging
    sheet.merge_cells('A1:H1')
    sheet.merge_cells('A2:H2')
    sheet.merge_cells('A4:C4')
    sheet.merge_cells('D4:F4')
    sheet.cell(4, 7).value = sheet.cell(5, 7).value
    sheet.merge_cells('G4:G5')
    sheet.cell(4, 8).value = sheet.cell(5, 8).value
    sheet.merge_cells('H4:H5')
    # Alignment
    i = 0

    while i < (last_index+5):
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.alignment = Alignment(horizontal='center', vertical='center')
        i += 1
    # Borders
    i = 0
    thin = Side(border_style='thin', color='000000')
    white_border = Side(border_style='thin', color='FFFFFF')

    while i < (last_index + 5):
        # 3rd row
        if i == 2:
            for c in sheet['A1:H' + str(last_index + 5)][i]:
                c.border = Border(left=white_border, right=white_border)
            i += 1
        for c in sheet['A1:H' + str(last_index + 5)][i]:
            c.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        i += 1
    # Column widths
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 15
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 50

In [124]:
if df_cis_filtered.shape[0] != 0:
    wb.save(root_path + r'/Output/CIS Exception Report (' +
            filtered_data_list[0].split()[0] + ').xlsx')

In [125]:
# Current date and time
e = datetime.datetime.now()
# Modify log file with information of interest
with open(root_path + str(Path(r'/Log.txt')), 'a') as f:
    f.write('\n')
    f.write('User: {}'.format(socket.gethostname()))
    f.write('\n')
    f.write('%s/%s/%s, %s:%s:%s' % (e.month, e.day, e.year,
                                    e.hour, e.minute, e.second))
    f.write('\n')
    f.write('Execution Time = {} seconds'.format(round(time.time()-start_time)))

In [126]:
# df_cis_report.iloc[:, 0] = df_cis_report.iloc[:, 0].values[::-1]

In [127]:
# First two rows of data are not being recognized for analysis
# Need to get correct direction of when survey was taken